In [1]:
#Import Python Packages
#from google.colab import drive
#drive.mount('/content/drive/')

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import f_classif,chi2
from sklearn.preprocessing import Binarizer, scale


#Read training data file
trainfile = r'/gdrive/My Drive/CIS_508/Datasets/Santander Customer Satisfaction - TRAIN.csv'
train = pd.read_csv(trainfile)

#Read test data file
testfile = r'/gdrive/My Drive/CIS_508/Datasets/Santander Customer Satisfaction - TEST-Without TARGET.csv'
test = pd.read_csv(testfile)

train.head()
#print("=======")
test.head()


#replace the values -999999 with the most frequent value 2 from the column

train = train.replace(-999999,2)
train.var3.value_counts()[:10]

def identify_constant_features(dataframe):
    count_uniques = dataframe.apply(lambda x: len(x.unique()))
    constants = count_uniques[count_uniques == 1].index.tolist()
    return constants

constant_features_train = set(identify_constant_features(train))

print('There were {} constant features in TRAIN dataset.'.format(
        len(constant_features_train)))


# Drop the constant features
final_train=train.drop(constant_features_train, inplace=True, axis=1)


#Select just Target Column
y_train = train.iloc[:, -1]
X_train = train.iloc[:,:-1]
X_test=test

x=train.values
test=test.values
y=final_train.values

X_train, X_test, y_train, y_test =train_test_split(x, y, random_state=1301, stratify=y_train, test_size=0.4)

#Create Decision Tree Classifier
clf=DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=100,
                 min_samples_split=20,max_features=295,max_leaf_nodes=3750)
              
#Apply Classifier on Train and Target
clf.fit(X_train,y_train)

#Get Class Prediction as a data frame with header as Prediction
pred=pd.DataFrame(clf.predict(X_train),columns=["Prediction"])

pred.head()

pred.Prediction.value_counts()

There were 34 constant features in TRAIN dataset.


AttributeError: ignored

In [0]:
print(train.shape)
print(test.shape)


(76020, 337)
(75818, 370)


In [0]:
pred_Probability =pd.DataFrame(clf.predict_proba(X_train))

pred_Probability.head()

,0,1
0,1.000000,0.000000
1,1.000000,0.000000
2,0.842105,0.157895
3,1.000000,0.000000
4,0.894737,0.105263


In [0]:
#Print Classification Report
print(classification_report(y_train,pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99     43807
           1       0.82      0.44      0.58      1805

    accuracy                           0.97     45612
   macro avg       0.90      0.72      0.78     45612
weighted avg       0.97      0.97      0.97     45612



In [0]:


pred=pd.DataFrame(clf.predict(X_test),columns=["TARGET"])

pred.head()


,TARGET
0,0
1,0
2,0
3,0
4,0


In [0]:
pred_Probability =pd.DataFrame(clf.predict_proba(X_test))

pred_Probability.head()

,0,1
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0


In [0]:
#submission file
pd.concat([test['ID'],pred['TARGET']],axis=1).to_csv("/gdrive/My Drive/CIS_508/Assignments/submission-Parameter2.csv", index = None)
